In [1]:
class Vertex:
    
    """Constructor for Vertex"""
    def __init__(self, n, cons=None):
        self.name = n
        if(cons == None):
            self.connections = []
        else:
            self.connections = cons

    """ Returns a Vertex's connections """
    def getConnections(self):
        return self.connections

    """ Returns a Vertex's name"""
    def getName(self):
        return self.name

    """ Adds a connection to the connections list of 
        this vertex"""
    def addConnections(self, t):
        self.connections.append(t)
    
    """ Removes a connection from the list. Unused in
        this notebook, but if a UI was to be implemented,
        being able to remove a node would be useful."""
    def removeConnection(self, t):
        self.connections.remove(t)

    """ Updates a connection value"""    
    def updateConnection(self, c1, c2):
        index = self.connections.index(c1)
        self.connections[index][0] = c2
    
    """ Nicely formats this Vertex's string"""
    def __str__(self):
        return "" + self.name

    __repr__ = __str__

In [8]:
class Graph:
    
    """ Constructor """
    def __init__(self):
        self.vertex_list = []
        self.edge_list_final = []
        self.vertex_dict = {}
    
    """Adds a vertex to the vertex_list of this Graph"""
    def addVertex(self, v):
        # Adds a vertex to the graph's overall vertex list
        self.vertex_list.append([v])
    
    """ This neatens up the edge list at the end
        It may be the case that duplicate edges were added
        In this example, for the first iteration, 
        A and D both choose the path connecting them
        This just neatens it for printing """
    def tidyFinalEdgeList(self):
        to_delete = []
        for i in self.edge_list_final:
            for j in self.edge_list_final:
                if i is not j and set(i) == set(j):
                    self.edge_list_final.remove(j)
                    
    """ This adjust the edge values. For example:
        Vertices A, B, C all are connected. On the first
        iteration, A and B are connected. C has a connection to A, 
        but A no longer exists, it is [A,B] now. This readjusts the
        connections of any point to it's value, by checking against
        a dictionary of changes"""
    def repointEdges(self, vertex_list):
        for i in vertex_list:
            for j in i[0].getConnections():
                val_to_check = j[0]
                while [val_to_check] not in vertex_list:
                    val_to_check = self.vertex_dict[val_to_check]
                i[0].updateConnection(j, val_to_check)

    """ This calculates the best edge (minimum weight) of a vertex"""
    def getBestEdge(self, v):
        #This is to handle the initial loop - 
        # items can come in as [Vertex] or as [Vertex, weight]
        # In a more standard OOP approach, this would be broken out
        # into a Vertex, Edge, and Graph class, but to make it more
        # pythonic the structure for an Edge is just [Vertex, weight]
        if isinstance(v[0], Vertex):
            v = [v]
        i = 0
        while True:
            # This would've been easier if Python 3 had an easy-to-use
            # Integer.MAX_VALUE, like Java, but instead the first 
            # item to compare against is the first case of a connection
            # that hasn't already been added in
            if v[0][0].getConnections()[i][0] not in v[0]:
                bestEdge = v[0][0].getConnections()[i]
                break
            i += 1
        # Now we have the first comparable, iterate through the 
        # list of connections and find the one with the minimum 
        # edge weight
        for i in v[0]:
            for c in i.getConnections():
                if c[1] < bestEdge[1] and c[0] not in v[0]:
                    bestEdge = c
        # Add the value to the final displaying edge list
        self.edge_list_final.append(v[0] + bestEdge)
        return bestEdge
    
    """This removes duplicates from the vertex_list given"""
    def remove_duplicates(self, vertex_list):
        # A duplicate is a duplicate if v2 is not exactly v (e.g memory address match)
        # and the number of nodes reached is the same
        for v in vertex_list:
            for v2 in vertex_list:
                if v is not v2:
                    if set(v[0]) == set(v2[0]):
                        vertex_list.remove(v2)

    """ This combines trees after the best edges are chosen. 
        For example, if [A,B], and [B,C] are both new elements,
        they have to be combined to form the subtree [A,B,C]. 
        How this is handled ends up with the new subtree being [A,C] 
        with the correct weight, but all the edges are captured in
        edge_list_final"""
    def combineTrees(self, edge_list):
        to_combine = []
        to_delete = []
        to_add = []
        num_of_changes = 1
        # The condition is if the subtrees (e.g. in the first round, individual paths)
        # have all been appropriately connected
        while num_of_changes != 0:
            to_combine = []
            to_delete = []
            to_add = []
            num_of_changes = 0
            for e in edge_list:
                for e2 in edge_list:
                    if e is not e2 and list(set(e[0]) & set(e2[0])):
                        if set(e[0]) != set(e2[0]):
                            to_combine += [[e, e2]]
                            to_delete += [e]
                            to_delete += [e2]
            # This logic somewhat gets repeated when the loop
            # finishes - this is the internal combining to make subtrees
            # not to make the new vertex containing the subtree
            for c in to_combine:
                num_of_changes += 1
                if list(set(c[0][0]) & set(c[1][0])):
                    new_vertex = [list(set(c[0][0] + c[1][0])),
                                  c[0][1]+c[1][1]]
                to_add += [new_vertex]
            self.remove_duplicates(to_add)
            for i in to_delete:
                try:
                    edge_list.remove(i)
                except ValueError:
                    pass
            edge_list += to_add
            self.remove_duplicates(edge_list)
        return edge_list
    """ This trims the duplicate connections - [a,b,4] is the same as [b,a,4]
        So we don't need to add both to the new Vertex object"""
    def trimNewConnections(self, ver_list, connections):
        to_delete = []
        for i in connections:
            if i[0] in ver_list:
                to_delete.append(i)
        connections = [x for x in connections if x not in to_delete]
        return connections
    
    """ This returns all of a subtree's connections
        It iterates through all the nodes in a subtree
        and gets their connections
        Since this will cause duplicates, trimNewConnections
        is called right after """
    def getNewConnections(self, ver_list):
        newConnections = []
        for i in ver_list:
            newConnections += i.getConnections()
        return newConnections
    
    """This combines two nodes and formats them correctly"""
    def combineNodes(self, v1, v2):
        # This method is broken out to help with visibility
        # If v1 = [a] and v2 = [b,4], then the result is [a,b,4]
        # If v1 = [a,c] and v2 = [b,4], then the result is [a,b,c,4]
        # You can't iterate over an object, so it has to be handled differently
        if isinstance(v1, Vertex):
            return [v1] + [v2]
        elif type(v1) == list:
            combined = [v1[0]] + [v2[0]]
            return [combined, v2[1]]

    """This is the actual body of the algorithm"""
    def boruvka_algorithm(self):
        total_cost = 0
        # We want to stop when there's the MST
        # This assumes a minimum spanning tree is possible, of course
        while len(self.vertex_list) != 1:
            best_edges = []
            # For every vertex in the list, get that vertex's minimum edge
            for v in self.vertex_list:
                bestEdge = self.getBestEdge(v)
                best_edges += [self.combineNodes(v, bestEdge)]
            # Now that there's the list of best edges, the list has to be 
            # be combined so a->b and b->c become a->c with the correct
            # weight. The new_edge_list represents the subtrees within
            # a graph.
            new_edge_list = self.combineTrees(best_edges)
            new_vertex_list = []
            for i in new_edge_list:
                # This creates the actual vertices out of the list of 
                # subtrees
                new_vertex_name = "<->".join(element.getName() for
                                             element in i[0])
                new_connections = self.getNewConnections(i[0])
                new_connections = self.trimNewConnections(i[0],
                                                          new_connections)
                new_vertex = Vertex(new_vertex_name,
                                    new_connections)
                for j in i[0]:
                    self.vertex_dict[j] = new_vertex
                new_vertex_list += [[new_vertex]]
            # After the new vertices have been made, references to nodes
            # within the subtree need to be redirected to the subtree itself
            self.repointEdges(new_vertex_list)
            self.vertex_list = new_vertex_list
        self.tidyFinalEdgeList()
        for e in self.edge_list_final:
            total_cost += e[2]
        print(f"Final Combined Vertex Output:{self.vertex_list[0]}, "
              f"\ncost:{total_cost}, \nedges:\n{self.edge_list_final}")
        print("The Final Combined Vertex Output is not the correct tree diagram output, "
              "just proving they are all connected. ")

The graph used for this demonstration is this:
![diagram](img/Algorithm%20Diagram.png)

In [9]:
gr = Graph()

#Initialise the Graph's vertices
a = Vertex("a")
b = Vertex("b")
c = Vertex("c")
d = Vertex("d")
e = Vertex("e")
f = Vertex("f")
g = Vertex("g")

#Add the connections in form [destination, weight]
a.addConnections([b, 7])
a.addConnections([d, 4])

b.addConnections([a, 7])
b.addConnections([c, 11])
b.addConnections([d, 9])
b.addConnections([e, 10])

c.addConnections([b, 11])
c.addConnections([e, 5])

d.addConnections([a, 4])
d.addConnections([b, 9])
d.addConnections([f, 6])
d.addConnections([e, 15])

e.addConnections([b, 10])
e.addConnections([c, 5])
e.addConnections([d, 15])
e.addConnections([f, 12])
e.addConnections([g, 8])

f.addConnections([d, 6])
f.addConnections([e, 12])
f.addConnections([g, 13])

g.addConnections([e, 8])
g.addConnections([f, 13])

#Add all the vertices to the graph
gr.addVertex(a)
gr.addVertex(b)
gr.addVertex(c)
gr.addVertex(d)
gr.addVertex(e)
gr.addVertex(f)
gr.addVertex(g)

#Run the algorithm
gr.boruvka_algorithm()

Final Combined Vertex Output:[c<->g<->e<->a<->f<->b<->d], 
cost:40, 
edges:
[[a, d, 4], [b, a, 7], [c, e, 5], [f, d, 6], [g, e, 8], [c<->g<->e, a<->f<->b<->d, 10]]
The Final Combined Vertex Output is not the correct tree diagram output, just proving they are all connected. 
